### API Key 가져오기

In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_O


### 문제 1-1 기본 Chain 만들기 - AI 요리사

- 문제 설명 :
사용자가 재료를 입력하면 그 재료로 만들 수 있는 요리를 추천해주는 간단한 AI 요리사를 만들어보세요.

요구사항
1. PromptTemplate을 사용하여 프롬프트 작성
2. 사용자가 입력한 재료를 받아서 만들 수 있는 요리 추천
3. ChatOpenAI 모델 사용
4. StrOutputParser로 결과를 문자열로 출력
5. LCEL(|) 문법을 사용하여 체인 연결

- 구현 조건 : 
입력: 재료명 (예: "토마토, 양파, 치즈")
출력: 추천 요리와 간단한 레시피


In [ ]:

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 1. 컴포넌트 정의
prompt = PromptTemplate.from_template("당신은 요리사입니다.  <Question>: {input}를 이용한 추천요리와 간단한 레시피를 알려주세요")

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# chain 연결 (LCEL)
chain = prompt | llm | StrOutputParser()

# 스트리밍 출력을 위한 요청
answer = chain.stream({"input": "토마토, 양파, 치즈"})
# 스트리밍 출력
#print(answer)

for token in answer:
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

### 문제 1-2 Multi Chain 만들기 - 여행지 정보 시스템
- 문제 설명 :
    사용자가 특정 도시나 국가를 입력하면, 해당 지역의 대표적인 관광 명소를 추천하고 그 명소에 대한 상세 정보(역사, 특징, 방문 팁 등)를 알려주는 2단계 체인을 구현해 보세요.
- 요구사항
1. 1단계 체인: 여행지(도시/국가)를 입력받아 대표 명소 1가지 추천
2. 2단계 체인: 추천받은 명소의 상세 정보 제공
3. ChatPromptTemplate 사용 
: “system”  과 “user” 메시지를 지정합니다.
4. 두 체인을 LCEL로 연결
5. 각 단계의 결과를 모두 출력하여 과정 확인
- 구현 조건
    - 입력: 여행지 (예: "로마", "뉴욕", "도쿄")
    - 1단계 출력: 추천 명소 이름
    - 2단계 출력: 명소에 대한 상세 정보 (역사, 특징, 방문 팁 등)



In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from pprint import pprint


prompt1 = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 여행유튜버입니다.") , 
     ("user", "{place} 에서 대표명소 한 곳을 추천해주세요") ]
)
prompt2 = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 여행유튜버입니다.") , 
     ("user", "{landmark}가 어딘지 알려주고 이 곳에 대한 상세정보를 알려주세요") ]
)

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1", 
    model="moonshotai/kimi-k2-instruct-0905", 
    temperature=0.7
)

chain1 = prompt1 | llm | StrOutputParser()

chain2 = (
    {"landmark": chain1}  #
    | prompt2
    | llm
    | StrOutputParser()
)

response = chain2.invoke({"place": "프랑스"})
print("\n🔹 명소 소개:")
pprint(response)


🔹 명소 소개:
('🇫🇷 유튜버들이 가장 많이 “깜빡” 하는 포인트  \n'
 '→ **에펠탑 2층 전망대(The Eiffel Tower 2nd floor)**  \n'
 '여기서 밤 10시(여름 11시) 스파클 쇼를 ‘직접’ 담으려면 반드시 **2층 표**를 사고, **22:00 전에 2층에 올라가 '
 '있어야** 합니다. 1층은 유리바닥·크리스마스 아이스링장이 있어 분위기 샷 좋지만, 스파클은 각도가 너무 각져서 “반짝”이 작게 '
 '찍혀요.\n'
 '\n'
 '---\n'
 '\n'
 '### 📍2층 전망대 촬영 가이드\n'
 '\n'
 '#### 1. 티켓 종류\n'
 '- **Stairs 2nd floor** : 계단만, €11.80 (내려올 때 1층 내부 촬영 가능)  \n'
 '- **Lift 2nd floor** : 엘리베이터直达, €18.80  \n'
 '- **Summit(정상)** : 2층 경유, €28.30  \n'
 '  ※ 정상은 줄 1~2시간 더 서고, 2층보다 스파클 시야가 오히려 넓게 안 잡히므로 **2층만** 추천\n'
 '\n'
 '#### 2. 예약 전략\n'
 '- 공식 사이트(www.toureiffel.paris) → “Buy tickets” → 날짜 선택 → **20:30~21:30 슬롯** '
 '예약  \n'
 '- 한 달 전 오전 9시(파리 시간)에 오픈 → **2~3분 내 매진**. 꼭 2~3일치 백업 날짜 준비  \n'
 '- 예약 없이 현장 구매 = 평균 1.5~2시간 대기, 밤 10시 쇼 놓침 확률 90%\n'
 '\n'
 '#### 3. 촬영 장비 규정\n'
 '- 삼각대·셀카봉·드론 전면 금지(보안검색대서 압수)  \n'
 '- 고프로·스마트폰·작은 짐벌(Osmo Pocket 등) OK  \n'
 '- 2층 난간 유리칸막이 높이 1.2m → **렌즈를 유리 틈에 최대한 밀착** → 반사 방지  \n'
 '- 추천 세팅  \n'
 '  - 4K 30p, ISO 100~400, 수동 초점 무한대, WB

### 문제1-3  FewShotPromptTemplate과 시스템 메시지 활용 

- 뉴스 키워드 추출기
1) 문제 설명
    - FewShotPromptTemplate을 사용하여 뉴스 기사에서 핵심 키워드 3개를 추출하는 시스템을 구현해보세요. 주어진 예시들을 참고하여 일관된 형식으로 키워드를 추출해야 합니다.
2) 요구사항
    - FewShotPromptTemplate 사용
    - 최소 3개의 예시(examples) 포함
    - 뉴스 텍스트에서 핵심 키워드 3개 추출
    - 일관된 출력 형식 유지
    - 다양한 분야의 뉴스로 테스트
